# 7day 정답

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import RobustScaler
import pandas as pd
# 랜덤 시드 설정 (재현성을 위해)
torch.manual_seed(1)
# 가상화폐 데이터 정의
X_data = pd.DataFrame({"BITCOIN":[82043000, 81351000, 83825000, 181939],  # BITCOIN 데이터
                    "SOLANA":[196800, 196300, 205800, 181939],  # SOLANA 데이터
                    "DOZICOIN":[152, 150, 156, 181939],  # DOZICOIN 데이터
                    "RIPPLE":[610, 608, 622, 181939],  # RIPPLE 데이터
                    "PEPECOIN":[0.02, 0.02, 0.02, 181939]})  # PEPECOIN 데이터
X_data=X_data.transpose()

In [ ]:



# 로버스트 스케일러 적용
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_data)
X_train = torch.FloatTensor(X_train_scaled)
y_train = torch.FloatTensor([[82601000], [202200], [153], [621], [0.02]])  # Y 데이터

# 모델 파라미터 초기화
W = torch.zeros(4, requires_grad=True)  # 가중치 초기화 (4개)
b = torch.zeros(1, requires_grad=True)  # 편향 초기화 (1개)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_vaild, y_train, y_vaild = train_test_split(X_train, y_train , test_size = 0.2, shuffle=True)

In [ ]:
# 옵티마이저 정의
optimizer = optim.SGD([W, b], lr=0.01)  # 확률적 경사 하강법(SGD) 사용
# 모델 학습
epochs = 100
# 미니배치 크기 설정
batch_size = 2
for epoch in range(epochs + 1):
    for i in range(0, len(X_train), batch_size):
        batch_x = X_train[i:i+batch_size]  # 미니배치 입력 데이터
        batch_y = y_train[i:i+batch_size]  # 미니배치 출력 데이터
        optimizer.zero_grad()  # 옵티마이저의 기울기 초기화
        # 선형 회귀 모델
        predict = batch_x * W + b  # 예측값 계산
        # 비용 함수 계산 (Mean Square Error)
        cost = torch.mean((predict - batch_y) ** 2)  # 실제값과 예측값 차이의 제곱 평균
        # 역전파를 통한 모델 업데이트
        cost.backward()  # 비용 함수에 대한 기울기 계산
        optimizer.step()  # 옵티마이저를 통해 가중치와 편향 업데이트
    # 100번째 에폭마다 로그 출력
    if epoch % 100 == 0:
        print(
            f'Epoch {epoch:4d}/{epochs} W: {W[0]}, {W[1]}, {W[2]} {W[3]} b: {b.item():.3f} Cost: {cost.item():.6f}'
        )
# 질문 포인트
# 왜 W가 0.0 일까?
# 미니배치는 정확히 뭘까?

Epoch    0/100 W: 37346.36328125, 37346.21875, 37346.96875 0.0 b: 44024.324 Cost: 1985843200.000000
Epoch  100/100 W: 64611.7734375, 64611.62890625, 64611.73828125 0.0 b: 39524.051 Cost: 1595085696.000000


# 8 day

## 실습1) 다음 조건을 만족하는 다중 선형회귀 학습을 짜시오

1. 데이터는 아래 데이터 그대로 사용할 것
2. 그에 맞는 다중 선형회귀를 클래스 형태로 짤 것.
3. 미니배치사이즈 4
4. 에폭 10
5. 옵티마이저, loss 마음대로 선택. 단, 성능은 신경쓰지 말 것.
6. import는 마음대로 넣을 것.
7. x_test 변수를 이용해 예측해서 결과를 출력하고 기울기함수를 해제하시오.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
x_train  =  torch.FloatTensor([[73,  80,  75, 88],
                               [93,  88,  93, 87],
                               [89,  91,  90, 95],
                               [96,  98,  93, 95],
                               [73,  66,  70, 77],
                               [77,  82,  75, 86],
                               [88,  75,  91, 90]])
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142], [151], [155]])

In [ ]:
x_test =  torch.FloatTensor([[82, 80, 77, 85]])


## 실습1) 제출 답

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

x_train  =  torch.FloatTensor([[73,  80,  75, 88],
                               [93,  88,  93, 87],
                               [89,  91,  90, 95],
                               [96,  98,  93, 95],
                               [73,  66,  70, 77],
                               [77,  82,  75, 86],
                               [88,  75,  91, 90]])
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142], [151], [155]])

dataset = TensorDataset(x_train, y_train) # 미니배치로 짜르기 전에, 데이터와 정답을 합친 후에 짤라야 한다.

dataloader = DataLoader(dataset, batch_size=2, shuffle=True,  drop_last=False) # 자, 그러면 마지막은 어떻게 될까?

# 모델을 선언 및 초기화. 단순 선형 회귀이므로 input_dim=3, output_dim=1.
# model = nn.Linear(4,1) # y= wx + wx + wx + b

class MultivariateLinearRegression(nn.Module):
    def __init__(self):
        super().__init__() # nn.Module 상속, 이와 같은 코드 super(MultivariateLinearRegression, self).__init__()
        self.linear = nn.Linear(4, 1) # 다중 선형 회귀이므로 input_dim=3, output_dim=1.

    def forward(self, x): # 순전파
        return self.linear(x)

# 가중치와 편향 선언
# W = torch.zeros((4, 1), requires_grad=True)    # 피처가 4개니까, y = wx + wx + wx + b
# b = torch.zeros(1, requires_grad=True)

model = MultivariateLinearRegression()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 2
for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    #print(batch_idx) # 미니배치 번호
    #print(samples) # 미니배치 내용물 확인

    x_train, y_train = samples # 정답이랑 답이 묶여있으니까 쪼개줘야 하지 않는가?
    print(x_train)
    print(y_train)
    prediction = model(x_train)
    cost = F.mse_loss(prediction, y_train)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))

# 임의의 데이터
x_test =  torch.FloatTensor([[82, 80, 77, 85]]) # 데이터 X 피처,, 1개를 때려박을때도 2차원으로 넣어야함.

# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(x_test)
print("훈련 후 입력이 82, 80, 77, 85일 때의 예측값 :", pred_y) # 여기서 문제, 왜 predict 값이 2차원일까요?

마지막에 임의 데이터를 모델로 예측할 때 평가 모드로 안했다.

## 실습1) 공개 답

In [103]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
x_train  =  torch.FloatTensor([[73,  80,  75, 88],
                               [93,  88,  93, 87],
                               [89,  91,  90, 95],
                               [96,  98,  93, 95],
                               [73,  66,  70, 77],
                               [77,  82,  75, 86],
                               [88,  75,  91, 90]])
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142], [151], [155]])
dataset = TensorDataset(x_train, y_train) # 미니배치로 짜르기 전에, 데이터와 정답을 합친 후에 짤라야 한다.
dataloader = DataLoader(dataset, batch_size=4, shuffle=True,  drop_last=False) # 자, 그러면 마지막은 어떻게 될까?

class MultivariateLinearRegression(nn.Module):
    def __init__(self):
        super().__init__() # nn.Module 상속, 이와 같은 코드 super(MultivariateLinearRegression, self).__init__()
        self.linear = nn.Linear(4, 1) # 다중 선형 회귀이므로 input_dim=4, output_dim=1.
    def forward(self, x): # 순전파
        return self.linear(x)

model = MultivariateLinearRegression()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 2
for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    #print(batch_idx) # 미니배치 번호
    #print(samples) # 미니배치 내용물 확인
    x_train, y_train = samples # 정답이랑 답이 묶여있으니까 쪼개줘야 하지 않는가?
    #print(x_train)
    #print(y_train)
    prediction = model(x_train)
    cost = F.mse_loss(prediction, y_train)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))
    
x_test =  torch.FloatTensor([[82, 80, 77, 85]])
model.eval()
with torch.no_grad():
  print("model prediction: ", round(model(x_test).detach()[0].tolist()[0],4))

Epoch    0/2 Batch 1/2 Cost: 18238.384766
Epoch    0/2 Batch 2/2 Cost: 2752.331299
Epoch    1/2 Batch 1/2 Cost: 782.412598
Epoch    1/2 Batch 2/2 Cost: 30.798666
Epoch    2/2 Batch 1/2 Cost: 86.837219
Epoch    2/2 Batch 2/2 Cost: 60.579792
model prediction:  160.4958


## 실습 2) 엉망인 코드를 고쳐보세요

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Dataset 상속
class CustomDataset(Dataset):
  def __init__(self):
    self.x_data = [[73, 80, 75],
                   [93, 88, 93],
                   [89, 91, 90],
                   [96, 98, 100],
                   [73, 66, 70]]
    self.y_data = [[152], [185], [180], [196], [142]]

  def __len__(self):
    return len(self.x_data)

  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])
    return x, y
  

class MultivariateLinearRegression(nn.Module):
def __init__(self):
    super().__init__()
    self.linear = nn.Linear(  ,  ) #수정

def forward(self, x):
    return self.linear(x)
    
dataset = CustomDataset()
dataloader = DataLoader(dataset, minibatch_size=2, suffle=True) #수정
model = MultivariateLinearRegression()
optimizer = torch.optim.SGD(model, rl=1e-5) #수정

nb_epochs = 20
for epoch in range(nb_epocs + 1): #수정
  for batch_idx, samples in eumerate(dataloader): #수정
    x_train, y_train = samples

    prediction = model(x_train)

    cost = F.mse_loss(      , y_train) #수정

    optimizer.zero_cola() #수정
    cost.backyard() #수정
    optimizer.stop() #수정

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))

## 실습2) 정답 제출

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Dataset 상속
class CustomDataset(Dataset):
  def __init__(self):
    self.x_data = [[73, 80, 75],
                   [93, 88, 93],
                   [89, 91, 90],
                   [96, 98, 100],
                   [73, 66, 70]]
    self.y_data = [[152], [185], [180], [196], [142]]

  def __len__(self):
    return len(self.x_data)

  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])
    return x, y

class MultivariateLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,1) #수정

    def forward(self, x):
        return self.linear(x)

dataset = CustomDataset()
dataloader = DataLoader(dataset) #수정
model = MultivariateLinearRegression()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) #수정

nb_epochs = 20
for epoch in range(nb_epochs + 1): #수정
  for batch_idx, samples in enumerate(dataloader): #수정
    x_train, y_train = samples

    prediction = model(x_train)

    cost = F.mse_loss(prediction, y_train) #수정

    optimizer.zero_grad() #수정
    cost.backward() #수정
    optimizer.step() #수정

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))

## 실습3) MLP로 MNIST 분류

In [104]:
import torch
import torch.nn as nn
from torch import optim
import numpy as np

In [105]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False)

In [106]:
mnist

{'data': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 'target': array(['5', '0', '4', ..., '4', '5', '6'], dtype=object),
 'frame': None,
 'categories': {'class': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']},
 'feature_names': ['pixel1',
  'pixel2',
  'pixel3',
  'pixel4',
  'pixel5',
  'pixel6',
  'pixel7',
  'pixel8',
  'pixel9',
  'pixel10',
  'pixel11',
  'pixel12',
  'pixel13',
  'pixel14',
  'pixel15',
  'pixel16',
  'pixel17',
  'pixel18',
  'pixel19',
  'pixel20',
  'pixel21',
  'pixel22',
  'pixel23',
  'pixel24',
  'pixel25',
  'pixel26',
  'pixel27',
  'pixel28',
  'pixel29',
  'pixel30',
  'pixel31',
  'pixel32',
  'pixel33',
  'pixel34',
  'pixel35',
  'pixel36',
  'pixel37',
  'pixel38',
  'pixel39',
  'pixel40',
  'pixel41',
  'pixel42',
  'pixel43',
  'pixel44',
  'pixel45',
  'pixel46',
  'pixel47

In [107]:
mnist.data[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [108]:
mnist.target[0]

'5'

In [109]:
X = mnist.data / 255  # 0-255값을 [0,1] 구간으로 정규화
y = mnist.target.astype(np.int8)

In [110]:
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

ds_train = TensorDataset(X_train, y_train)
ds_test = TensorDataset(X_test, y_test)

loader_train = DataLoader(ds_train, batch_size=64, shuffle=True)
loader_test = DataLoader(ds_test, batch_size=64, shuffle=False)

In [111]:
class MultivariateLinearRegression(nn.Module):
    def __init__(self):
        super().__init__() # nn.Module 상속, 이와 같은 코드 super(MultivariateLinearRegression, self).__init__()
        self.linear1 = nn.Linear(28*28*1, 100) # input_layer = 784, hidden_layer1 = 100
        self.activate1 = nn.ReLU()
        self.linear2 = nn.Linear(100, 100) # hidden_layer2 = 100, hidden_layer3 = 100
        self.activate2 = nn.ReLU()
        self.linear3 = nn.Linear(100, 10) # hidden_layer3 = 100, output_layer = 10

    def forward(self, x): # 순전파
        out1 = self.linear1(x)
        out2 = self.activate1(out1)
        out3 = self.linear2(out2)
        out4 = self.activate2(out3)
        out5 = self.linear3(out4)
        return out5

model = MultivariateLinearRegression()

In [112]:
loss_fn = nn.CrossEntropyLoss() # 이 비용 함수는 소프트맥스 함수를 포함하고 있음.
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(3):

  model.train()  # 신경망을 학습 모드로 전환
  # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행

  for data, targets in loader_train:
    optimizer.zero_grad()  # 경사를 0으로 초기화
    outputs = model(data)  # 데이터를 입력하고 출력을 계산
    loss = loss_fn(outputs, targets)  # 출력과 훈련 데이터 정답 간의 오차를 계산
    loss.backward()  # 오차를 역전파 계산
    optimizer.step()  # 역전파 계산한 값으로 가중치를 수정


    model.eval()  # 신경망을 추론 모드로 전환
    correct = 0
    # 데이터로더에서 미니배치를 하나씩 꺼내 추론을 수행
    with torch.no_grad():  # 추론 과정에는 미분이 필요없음
        for data, targets in loader_test:
            outputs = model(data)  # 데이터를 입력하고 출력을 계산

            # 추론 계산
            _, predicted = torch.max(outputs.data, 1)  # 확률이 가장 높은 레이블이 무엇인지 계산
            correct += predicted.eq(targets.data.view_as(predicted)).sum()
            # 정답과 일치한 경우 정답 카운트를 증가

    # 정확도 출력
    data_num = len(loader_test.dataset)  # 데이터 총 건수
    print('테스트 데이터에서 예측 정확도: {}/{} ({:.0f}%)\n'.format(correct,
                                                   data_num, 100. * correct / data_num))

테스트 데이터에서 예측 정확도: 2546/14000 (18%)

테스트 데이터에서 예측 정확도: 4462/14000 (32%)

테스트 데이터에서 예측 정확도: 5884/14000 (42%)

테스트 데이터에서 예측 정확도: 8345/14000 (60%)

테스트 데이터에서 예측 정확도: 8352/14000 (60%)

테스트 데이터에서 예측 정확도: 9183/14000 (66%)

테스트 데이터에서 예측 정확도: 8890/14000 (64%)

테스트 데이터에서 예측 정확도: 9036/14000 (65%)

테스트 데이터에서 예측 정확도: 8927/14000 (64%)

테스트 데이터에서 예측 정확도: 9941/14000 (71%)

테스트 데이터에서 예측 정확도: 8904/14000 (64%)

테스트 데이터에서 예측 정확도: 9692/14000 (69%)

테스트 데이터에서 예측 정확도: 9872/14000 (71%)

테스트 데이터에서 예측 정확도: 9457/14000 (68%)

테스트 데이터에서 예측 정확도: 9828/14000 (70%)

테스트 데이터에서 예측 정확도: 10402/14000 (74%)

테스트 데이터에서 예측 정확도: 10856/14000 (78%)

테스트 데이터에서 예측 정확도: 11315/14000 (81%)

테스트 데이터에서 예측 정확도: 11161/14000 (80%)

테스트 데이터에서 예측 정확도: 10800/14000 (77%)

테스트 데이터에서 예측 정확도: 10615/14000 (76%)

테스트 데이터에서 예측 정확도: 10861/14000 (78%)

테스트 데이터에서 예측 정확도: 11321/14000 (81%)

테스트 데이터에서 예측 정확도: 11462/14000 (82%)

테스트 데이터에서 예측 정확도: 11308/14000 (81%)

테스트 데이터에서 예측 정확도: 11194/14000 (80%)

테스트 데이터에서 예측 정확도: 11397/14000 (81%)

테스트 데이터에서 예측 정확도